

<center>**Bachelor of Ecole Polytechnique**</center>
<center>Computational Mathematics, year 2, semester 1</center>
<center>Lecturer: Lucas Gerin <a href="mailto:lucas.gerin@polytechnique.edu">(send mail)</a></center>

# Symbolic computation 2: Linear recurrences


## Table of contents

- [Warm-up](#warmup)
- Solving linear recurrences with `Sympy`
  - [The function `rsolve`](#rsolve)
  - [Application: Fibonacci](#Fibo)
- [Two-dimensional recurrence: matrices with SymPy](#2d)
- [Bonus (from BX2023's test)](#Bonus)

In [2]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom2.css").read()
    return HTML(styles)
css_styling()

In [3]:
## loading python libraries

# necessary to display plots inline:
%matplotlib inline   

# load the libraries
import matplotlib.pyplot as plt # 2D plotting library
import numpy as np              # package for scientific computing  
from pylab import *

from math import *              # package for mathematics (pi, arctan, sqrt, factorial ...)
import sympy as sympy             # package for symbolic computation
from sympy import *


<a id="warmup"></a>
The aim of this Lab session is to use SymPy to solve automatically some simple recurrences. We first solve a particular case "by hand" and then use the SymPy function `rsolve`. 

## Warm-up

### Exercise 1. Solving a recurrence (almost) by hand

<div markdown=1 class="DoIt"> 

We want to find an explicit formula for the sequence $(u_n)$ defined by
\begin{equation}
\begin{cases}
u_0&=1,\\
u_{n}&=2u_{n-1} +3n^2. \qquad (\forall n\geq 1).
\end{cases}
\tag{$\star$}
\end{equation}
1. Let  $\alpha,a,b,c$ be four real parameters. Let $(w_n)_{n\geq 0}$ be the sequence defined by
$$
w_n=\alpha 2^n +an^2+bn+c.
$$
Use `SymPy` to find $\alpha,a,b,c$ such that for every $0\leq n\leq 3$,
$$
w_n=u_n.
$$
<i>To solve a system of equations with `SymPy` with unknowns $x,y$, write something like </i>
```
solve([x-y-2,3*y+x],[x,y])
```
2. Prove with `SymPy` that $(w_n)$ defined as above is the unique solution of ($\star$).

<i>(It might be helpful to define a function `w(n,alpha,a,b,c)` which computes $w_n$.)</i>

In [6]:
# ------------- Question 1 --------------
var('n',integer=True)
var('alpha a b c')
def w(n,alpha,a,b,c):
    return alpha*2**n +a*n**2 +b*n +c

Solutions=solve(
# Quantities which are to be zero:
[w(0,alpha,a,b,c)-1,   # initial condition
w(1,alpha,a,b,c)-2*w(0,alpha,a,b,c)-3*1**2, 
w(2,alpha,a,b,c)-2*w(1,alpha,a,b,c)-3*2**2, 
w(3,alpha,a,b,c)-2*w(2,alpha,a,b,c)-3*3**2, 
]
,
# Unknowns:
[alpha,a,b,c]
)
print('--------------')
print('Question 1:', Solutions)



--------------
Question 1: {alpha: 19, a: -3, b: -12, c: -18}


<div markdown=1 class="Answers"> 

1. We have four unknowns $\alpha,a,b,c$ and four equations
\begin{align*}
u_0&=1,\\
u_1&=2u_0+3\times 1^2,\\
u_2&=2u_1+3\times 2^2,\\
u_3&=2u_2+3\times 3^2,\\
\end{align*}
With the above script we obtain that if the formula is true then necessarily
$$
u_n=19\times 2^n -3n^2-12n-18.
$$


In [11]:
#-------------- Question 2 ----------------
print('--------------')
c_s=Solutions[c]
alpha_s=Solutions[alpha]
b_s=Solutions[b]
a_s=Solutions[a]
Expr=w(n,alpha_s,a_s,b_s,c_s)-2*w(n-1,alpha_s,a_s,b_s,c_s),
-3*n**2
print('Question 2:')
print('With these coefficients the simplification of w(n)-2w(n-1)-3n**2 gives for every n : '+str(simplify(Expr)))

#--------------- We check the solutions
print('--------------')
print('Safety check:')
print('First values of w_n: '+str([w(i,alpha_s,a_s,b_s,c_s) for i in range(10)]))

# In order to check our results:
def Recurrence(n):
    if n==0:
        return 1
    else:
        return 2*Recurrence(n-1)+3*n**2

print('First values (with recurrence formula) : '+str([Recurrence(n) for n in range(10)]))

--------------
Question 2:
With these coefficients the simplification of w(n)-2w(n-1)-3n**2 gives for every n : (19*2**n - 38*2**(n - 1) - 3*n**2 + 12*n + 6*(n - 1)**2 - 6,)
--------------
Safety check:
First values of w_n: [1, 5, 22, 71, 190, 455, 1018, 2183, 4558, 9359]
First values (with recurrence formula) : [1, 5, 22, 71, 190, 455, 1018, 2183, 4558, 9359]



<div markdown=1 class="Answers"> 

2. We first observe that eq.$(\star)$ has a unique solution since the sequence $(u_n)$ is a recurrence of order one: it is uniquely determined by $u_0$.
Besides, set
<br>
$$
w_n=19\times 2^n -3n^2-12n-18.
$$
<br>
Parameters are chosen so that $w_0=u_0=1$.
The above script shows that for every $n$ we have that $w_n-2w_{n-1}-3n^2=0$. Therefore $(w_n)$ satisfies the recurrence $(\star)$ and starts with the same initial value $w_0=1$. Therefore $w_n=u_n$ for every $n$.

<a id="rsolve"></a>
## Solving recurrences with SymPy: `rsolve`

### The function `rsolve`

We will use Sympy to obtain explicit formulas for some sequences defined by linear recurrences. More precisely, we will see how to obtain an explicit formula for $u_n$ in two cases:

1. **Linear recurrence of order one:** this is a sequence $(u_n)_{n\geq 0}$ is defined by
$$
\begin{cases}
u_0&=a, \\
u_{n}&=\alpha u_{n-1}+f(n),\qquad (n\geq 1),
\end{cases}
$$
where $a,\alpha$ are some given constants and $f$ is an arbitrary function.
2. **Linear recurrence of order two:** this is a sequence $(u_n)_{n\geq 0}$ is defined by
$$
\begin{cases}
u_0&=a, \\
u_1&=b,\\
u_{n}&=\alpha u_{n-1}+\beta u_{n-2}+f(n),\qquad (n\geq 2),
\end{cases}
$$
where $a,,b,\alpha,\beta$ are some given constants and $f$ is an arbitrary function.

Some known examples fit in this settings:
1. Geometric sequences: $u_0=a,u_n=ru_{n-1}$.
2. Arithmetic sequences: $u_0=a,u_n=u_{n-1}+r$.
3. The Fibonacci sequence: $F_1=1,F_2=1,F_{n}=F_{n-1}+F_{n-2}$.

The following script shows how to solve the two recurrences
\begin{align*}
u_0=5,\qquad u_{n}&=3u_{n-1},\\
v_0=1,\qquad v_{n}&=2v_{n-1}+n,\\
\end{align*}
using function `rsolve`.

In [7]:
# First example: a geometric sequence
u = Function('u')                 # declares the name of the sequence
n = symbols('n',integer=True)     # declares the variable
f = u(n)-3*u(n-1)                 # the expression which is to be zero
ExplicitFormula1=rsolve(f,u(n),
                        {u(0):5}) # initial condition

print('The formula for u(n) is '+str(ExplicitFormula1)+'')

# Second example with a non-linear term
print('-------')
v = Function('v')                 # declares the name of the sequence
n = symbols('n',integer=True)     # declares the variable
f = v(n)-2*v(n-1)-n           # the expression which is to be zero
ExplicitFormula2=rsolve(f,v(n),
                        {v(0):1}) # initial condition

print('The formula for v(n) is '+str(ExplicitFormula2)+'')



The formula for u(n) is 5*3**n
-------
The formula for v(n) is 2**n/2 + n/2 + 1/2


<a id="Fibo"></a>

### Exercise 2. A first example with `rsolve`

<div markdown=1 class="DoIt"> 

1. Use `SymPy` to solve the recurrence of the Fibonacci sequence and find an explicit formula for $F_n$.<br>
(To set up two initial conditions you must write `{u(1):1,u(2):1}`.)
2. **(Theory)** Use the formula obtained at Question 1 to prove that 
$$
\lim_{n\to +\infty} \frac{F_n}{F_{n-1}}= \varphi,
$$
where $\varphi=\frac{1+\sqrt{5}}{2}$ is the golden ratio.

In [16]:
u = Function('u')                 # declares the name of the sequence
n = symbols('n',integer=True)     # declares the variable
f = u(n)-u(n-1)-u(n-2)                 # the expression which is to be zero
ExplicitFormula=simplify(rsolve(f,u(n),{u(1):1,u(2):1}))

Formula2=latex(simplify(ExplicitFormula))
print('The formula for u(n) is '+str(Formula2)+'')


The formula for u(n) is \frac{2^{- n} \sqrt{5} \left(- \left(1 - \sqrt{5}\right)^{n} + \left(1 + \sqrt{5}\right)^{n}\right)}{5}


<div markdown=1 class="Answers">
1. We put the above answer in $\texttt{LateX}$:
$$
F_n=\frac{2^{- n}}{5} \sqrt{5} \left(\left(1 + \sqrt{5}\right)^{n} - \left(- \sqrt{5} + 1\right)^{n}\right).
$$
2. We obtain (by setting $\psi=(1-\sqrt{5})/2$):
\begin{align*}
\frac{F_{n}}{F_{n-1}}&= \frac{\tfrac{1}{\sqrt{5}}\varphi^{n} - \tfrac{1}{\sqrt{5}}\psi^n }{\tfrac{1}{\sqrt{5}}\varphi^{n-1} - \tfrac{1}{\sqrt{5}}\psi^{n-1} },\\
&=  \frac{\tfrac{1}{\sqrt{5}}\varphi - \psi \tfrac{1}{\sqrt{5}}(\psi/\varphi)^{n-1} }{\tfrac{1}{\sqrt{5}} - \tfrac{1}{\sqrt{5}}(\psi/\varphi)^{n-1} },\qquad \text{(we divide everything by $\varphi^{n-1}$.)}\\
&\to \varphi,
\end{align*}
since $|\psi/\varphi|<1$.

<div markdown=1 class="Rmk">

The output of `rsolve` is an <i>expression</i> which depends on the symbolic variable `n`. If we want to evaluate this expression (for instance for $n=10$) we must write:

In [ ]:
Value=ExplicitFormula.subs(n,10)
print('10th Fibonacci number = '+str(Value))
print('After simplification : '+str(simplify(Value)))


<a id="2d"></a>
# Two-dimensional recurrence: matrices with SymPy
### Exercise 3. A double linear recurrence


<div markdown=1 class="DoIt"> <b>Theory</b>

1. Prove by induction that there exist integers $a_n,b_n$ such that for every $n\geq 1$
$$
(1+\sqrt{2})^n=a_n+b_n\sqrt{2}.
$$
2. Find a $2\times 2$ matrix $A$ such that
$$
\binom{a_{n+1}}{b_{n+1}}=A\times \binom{a_{n}}{b_{n}}.
$$
3. Use `SymPy` to find an explicit formula for $a_n$.<br>
<i>(In `SymPy` matrices are defined by `A=Matrix([[a,b],[c,d]])`. To raise $A$ to some power just write `A**n`.)</i>
4. Find (with pen/paper) real numbers $c,R$ such that
$$
a_n \stackrel{n\to +\infty}{\sim } cR^n.
$$

<div markdown=1 class="Answers"> 
1. For $n=1$ this is true with $a_1=b_1=1$. Assume this holds for some $n\geq 1$.
\begin{align*}
(1+\sqrt{2})^{n+1}&=(1+\sqrt{2})\times(1+\sqrt{2})^{n}\\
&=(1+\sqrt{2})\times(a_n+b_n\sqrt{2})\\
&=a_n+b_n\sqrt{2}+a_n\sqrt{2}+2b_n\\
&=(a_n+2b_n)+(a_n+b_n)\sqrt{2}.
\end{align*}
Finally, one can write 
$$
(1+\sqrt{2})^{n+1}=a_{n+1}+b_{n+1}\sqrt{2},
$$
where
$$
a_{n+1}=a_n+2b_n,\qquad b_{n+1}=a_n+b_n,
$$
which are easily proved to be integers by induction.

2. By the above computation we have
$$
\begin{cases}
a_{n+1}&=a_n+2b_n,\\
b_{n+1}&=a_n+b_n,
\end{cases}
$$
which can be written as:
$$
\binom{a_{n+1}}{b_{n+1}}=
\begin{pmatrix}
1 & 2\\
1 & 1
\end{pmatrix}
\times \binom{a_{n}}{b_{n}}.
$$
By induction this implies that
$$
\binom{a_{n}}{b_{n}}=
\begin{pmatrix}
1 & 2\\
1 & 1
\end{pmatrix}^{n-1}
\times \binom{a_1}{b_1}
=\begin{pmatrix}
1 & 2\\
1 & 1
\end{pmatrix}^{n-1}
\times \binom{1}{1}.
$$

In [6]:
#Question 3
A=Matrix([[1,2],[1,1]])
var('n', integer=True)
Power=A**(n-1)
display(Power)

an=latex(simplify(Power[0,0]+Power[0,1]))
print(an)
bn=latex(simplify(Power[1,0]+Power[1,1]))
print(bn)


Matrix([
[                 (1 - sqrt(2))**(n - 1)/2 + (1 + sqrt(2))**(n - 1)/2, -sqrt(2)*(1 - sqrt(2))**(n - 1)/2 + sqrt(2)*(1 + sqrt(2))**(n - 1)/2],
[-sqrt(2)*(1 - sqrt(2))**(n - 1)/4 + sqrt(2)*(1 + sqrt(2))**(n - 1)/4,                  (1 - sqrt(2))**(n - 1)/2 + (1 + sqrt(2))**(n - 1)/2]])

\frac{\left(1 - \sqrt{2}\right)^{n}}{2} + \frac{\left(1 + \sqrt{2}\right)^{n}}{2}
\frac{\sqrt{2} \left(- \left(1 - \sqrt{2}\right)^{n} + \left(1 + \sqrt{2}\right)^{n}\right)}{4}


<div markdown=1 class="Answers">
3. We export the result in LateX:
\begin{align*}
a_n&=\frac{\left(1 - \sqrt{2}\right)^{n}}{2} + \frac{\left(1 + \sqrt{2}\right)^{n}}{2}\\
b_n&= \frac{\sqrt{2} \left(- \left(1 - \sqrt{2}\right)^{n} + \left(1 + \sqrt{2}\right)^{n}\right)}{4}
\end{align*}
(You can indeed check that $a_n+\sqrt{2}b_n=(1+\sqrt{2})^n$.)

4. As $\left|-\sqrt{2}+1\right|<1$, we have that $(-\sqrt{2}+1)^n$ tends to zero. It follows that
\begin{align*}
a_n&=\frac{1}{2} \left(1 + \sqrt{2}\right)^{n} +\mathrm{o}(1)\\
&\sim \frac{1}{2} \left(1 + \sqrt{2}\right)^{n}.
\end{align*}

 <a id='Bonus'></a>
### Exercise 4. Rational fractions. (Taken from BX2023's test)

<div markdown=1 class="DoIt"> 
We set
$$
u_1=\frac{1}{1+1}, \quad u_2=\frac{1}{1+\frac{1}{2+1}},\quad u_3=\frac{1}{1+\frac{1}{2+\frac{1}{3+1}}}, \quad u_4=\frac{1}{1+\frac{1}{2+\frac{1}{3+\frac{1}{4+1}}}}, \dots
$$
Using SymPy, write $u_{20}$ as a rational fraction $a/b$. (To check your result: the approximate value is $0.69777...$.)

In [9]:
# Tests
u1=Rational(1,2)
u2=Rational(1,1+Rational(1,3))
u3=Rational(1,1+Rational(1,2+Rational(1,4)))

def u_n(n):
    u=n+1
    for k in range(1,n):
        u=n-k+Rational(1,u)
    return Rational(1,u)

for n in range(1,21):
    print(n,'---',u_n(n))
    
    

1 --- 1/2
2 --- 3/4
3 --- 9/13
4 --- 37/53
5 --- 187/268
6 --- 1129/1618
7 --- 7933/11369
8 --- 63621/91177
9 --- 573561/821986
10 --- 5742571/8229836
11 --- 63224941/90609397
12 --- 759216193/1088053549
13 --- 9875036179/14152185188
14 --- 138308505777/198213712978
15 --- 2075328803577/2974210627873
16 --- 33214434676489/47600517297953
17 --- 564774524186833/809393860526194
18 --- 10167887629480051/14571878633638372
19 --- 193221133200680401/276910505412260141
20 --- 3864956170297235421/5538974690732597941


<div markdown=1 class="Answers"> 
We set
\begin{align*}
v_0&=21\\
v_1&=20+\frac{1}{v_0}=20+\frac{1}{20+1}\\
v_2&=19+\frac{1}{v_1}=19+\frac{1}{20+\frac{1}{20+1}}\\
& \dots\\
v_{20}&=1+\frac{1}{v_{19}}=1+\frac{1}{2+\frac{1}{\dots+\frac{1}{{\dots+\frac{1}{20+1}}}}}
\end{align*}
And we finally put $u_{20}=\frac{1}{v_{20}}$.

Therefore the above script computes:
$$
u_{20}=\frac{3864956170297235421}{5538974690732597941}.
$$